In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import imageio

# Function to update the frame in the tkinter window
def update_frame():
    frame = reader.get_next_data()  # Get the next frame from the webcam
    frame_image = Image.fromarray(frame)  # Convert to PIL Image
    imgtk = ImageTk.PhotoImage(image=frame_image)  # Convert to tkinter-compatible image
    label.imgtk = imgtk
    label.configure(image=imgtk)
    label.after(10, update_frame)  # Update after 10 milliseconds

# Create a tkinter window
root = tk.Tk()
root.title("Webcam Live Feed")

# Open the webcam using imageio
try:
    reader = imageio.get_reader("<video0>")
except Exception as e:
    print(f"Error: {e}")
    exit()

# Create a tkinter label to display the frames
label = tk.Label(root)
label.pack()

# Start the webcam frame update
update_frame()

# Start the tkinter event loop
root.mainloop()

# Close the reader when done
reader.close()


In [2]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import cv2
import face_recognition
import numpy as np
import requests
import tempfile
import os

def register_face_to_server(server_url, user_id, person_name, image_path):
    """
    Envia a imagem, user_id e o nome para o servidor usando multipart/form-data.
    """
    data = {'user_id': user_id, 'name': person_name}

    try:
        with open(image_path, 'rb') as image_file:
            files = {'image': image_file}
            response = requests.post(server_url, data=data, files=files)

        if response.status_code in [200, 201]:
            response_json = response.json()
            if 'message' in response_json:
                messagebox.showinfo("Cadastro", response_json['message'])
        else:
            messagebox.showerror("Erro", f"Falha ao cadastrar rosto: {response.text}")

    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao cadastrar o rosto: {str(e)}")

def detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url):
    net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    video_capture = cv2.VideoCapture(0)
    if not video_capture.isOpened():
        print("Não foi possível acessar a webcam.")
        return

    stop = False
    face_detected = False
    temp_image_path = None  # Para armazenar a imagem temporária capturada
    frozen_frame = None  # Frame congelado após a detecção
    frozen_face_locations = []  # Localizações do rosto congeladas
    frozen_face_encodings = []  # Codificações faciais congeladas

    def on_close():
        nonlocal stop, temp_image_path
        stop = True
        print("Fechando a janela e liberando a câmera...")
        if video_capture.isOpened():
            video_capture.release()
        if temp_image_path and os.path.exists(temp_image_path):
            try:
                os.remove(temp_image_path)  # Remover o arquivo temporário
            except PermissionError:
                print(f"Erro: O arquivo '{temp_image_path}' ainda está em uso.")
        root.destroy()

    root = tk.Tk()
    root.title("Webcam ao Vivo")

    label = tk.Label(root)
    label.pack()

    frame_controls = tk.Frame(root)
    frame_controls.pack()

    btn_register = tk.Button(frame_controls, text="Cadastrar Rosto", command=lambda: open_registration_form(server_register_url))
    btn_register.pack(side=tk.LEFT, padx=5)
    btn_register["state"] = tk.DISABLED  # Desabilitar até que um rosto seja detectado

    btn_recognize = tk.Button(frame_controls, text="Reconhecer Rosto", command=lambda: recognize_face(server_recognize_url))
    btn_recognize.pack(side=tk.LEFT, padx=5)
    btn_recognize["state"] = tk.DISABLED  # Desabilitar até que um rosto seja detectado

    btn_resume = tk.Button(frame_controls, text="Retomar Webcam", command=lambda: resume_webcam())
    btn_resume.pack(side=tk.LEFT, padx=5)
    btn_resume["state"] = tk.DISABLED  # Desabilitar até que o feed seja pausado

    root.bind('<Control-q>', lambda event: on_close())
    root.protocol("WM_DELETE_WINDOW", on_close)

    def resume_webcam():
        nonlocal face_detected, frozen_frame, frozen_face_locations, frozen_face_encodings
        face_detected = False
        frozen_frame = None
        frozen_face_locations = []
        frozen_face_encodings = []
        btn_register["state"] = tk.DISABLED
        btn_recognize["state"] = tk.DISABLED
        btn_resume["state"] = tk.DISABLED
        update_frame()

    def update_frame():
        nonlocal face_detected, temp_image_path, frozen_frame, frozen_face_locations, frozen_face_encodings
        if stop or face_detected:
            return

        ret, frame = video_capture.read()
        if not ret:
            print("Falha ao capturar imagem da webcam")
            return

        (h, w) = frame.shape[:2]

        # Pré-processar a imagem para a detecção de rosto
        blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
        net.setInput(blob)
        detections = net.forward()

        face_locations = []

        # Percorrer as detecções
        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]

            # Filtrar detecções fracas (threshold de confiança pode ser ajustado)
            if confidence > 0.5:
                # Obter as coordenadas da caixa delimitadora para o rosto detectado
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")

                # Adicionar à lista de localizações de rosto
                face_locations.append((startY, endX, endY, startX))

                # Desenhar a caixa delimitadora ao redor do rosto
                cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)

        # Converter o frame para RGB (compatível com tkinter)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_image = Image.fromarray(rgb_frame)
        imgtk = ImageTk.PhotoImage(image=frame_image)

        # Atualizar o label tkinter com a nova imagem
        label.imgtk = imgtk
        label.configure(image=imgtk)

        # Verificar se algum rosto foi detectado
        if len(face_locations) > 0:
            print("Rosto detectado!")

            # Obter as codificações faciais para os rostos detectados
            frozen_face_encodings = face_recognition.face_encodings(frame, face_locations)

            if len(frozen_face_encodings) > 0:
                # Salvar a imagem congelada
                _, temp_image_path = tempfile.mkstemp(suffix='.png')
                cv2.imwrite(temp_image_path, frame)
                os.close(_)  # Fechar o arquivo corretamente

                btn_register["state"] = tk.NORMAL
                btn_recognize["state"] = tk.NORMAL
                btn_resume["state"] = tk.NORMAL
            else:
                print("Falha ao gerar codificações faciais.")

        else:
            label.after(10, update_frame)

    def open_registration_form(server_register_url):
        nonlocal temp_image_path

        def on_submit():
            user_id = entry_user_id.get()
            person_name = entry_name.get()
            if user_id and person_name:
                if temp_image_path:
                    register_face_to_server(server_register_url, user_id, person_name, temp_image_path)
                else:
                    messagebox.showerror("Erro", "Nenhuma imagem de rosto capturada para cadastrar.")
                top.destroy()
            else:
                messagebox.showerror("Erro", "Por favor, preencha todos os campos.")

        top = tk.Toplevel(root)
        top.title("Cadastro de Rosto")

        tk.Label(top, text="User ID:").pack()
        entry_user_id = tk.Entry(top)
        entry_user_id.pack()

        tk.Label(top, text="Nome:").pack()
        entry_name = tk.Entry(top)
        entry_name.pack()

        tk.Button(top, text="Cadastrar", command=on_submit).pack()

    def recognize_face(server_recognize_url):
        nonlocal frozen_face_encodings
        if len(frozen_face_encodings) == 0:
            messagebox.showinfo("Reconhecimento", "Nenhum rosto detectado ou imagem congelada.")
            return

        # Preparar os dados para enviar ao servidor
        data_to_send = {
            'encodings': [encoding.tolist() for encoding in frozen_face_encodings]  # Converter arrays numpy para listas
        }

        # Debug: Print the data being sent
        print("Data being sent for recognition:", data_to_send)

        # Enviar as codificações faciais para o servidor
        try:
            response = requests.post(server_recognize_url, json=data_to_send)
            print("Server response status code:", response.status_code)  # Debug: Print the status code
            if response.status_code == 200:
                response_data = response.json()
                print("Server response data:", response_data)  # Debug: Print the response data
                messagebox.showinfo("Reconhecimento", response_data.get("message", "Nenhum dado retornado."))
            else:
                messagebox.showerror("Erro", f"Erro ao reconhecer o rosto. Código: {response.status_code}")
        except Exception as e:
            messagebox.showerror("Erro", f"Erro ao comunicar com o servidor: {e}")

    update_frame()
    recognize_face(server_recognize_url)
    root.mainloop()

# Testar a função com as URLs de cadastro e reconhecimento
server_register_url = 'http://127.0.0.1:5000/register'
server_recognize_url = 'http://127.0.0.1:5000/recognize'
detect_faces_from_webcam_and_send_to_server(server_register_url, server_recognize_url)

Rosto detectado!
Rosto detectado!
Rosto detectado!
Rosto detectado!
Data being sent for recognition: {'encodings': [[-0.13514047861099243, 0.09851565957069397, 0.0052752927877008915, -0.01761622354388237, -0.11115045845508575, -0.028671033680438995, -0.03168337419629097, -0.0631011426448822, 0.11115289479494095, -0.18317146599292755, 0.19750070571899414, -0.01836629956960678, -0.2162642478942871, 0.005952829960733652, -0.023471012711524963, 0.12907297909259796, -0.015407415106892586, -0.1575891375541687, -0.1625908613204956, -0.18622414767742157, -0.056877702474594116, 0.04019317030906677, -0.05890089273452759, 0.017366904765367508, -0.09238237142562866, -0.3155537545681, -0.0990133136510849, -0.05878031626343727, 0.07027134299278259, -0.10300156474113464, 0.02332380786538124, 0.05272933095693588, -0.05614638328552246, -0.0085330531001091, 0.06332607567310333, 0.06265760213136673, -0.042487695813179016, -0.03493478521704674, 0.25714871287345886, -0.014658117666840553, -0.06037754938006